In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Data
Year = [2017, 2018, 2019, 2020, 2021, 2022, 2023]
Water_Areas = [240, 190, 190, 180, 170, 228, 433]
Trees = [79071, 75514, 75394, 75492, 75929, 79514, 75152]
Flooded_Vegetation_Areas = [251, 163, 171, 38, 3, 14, 22]
Crops = [3328, 3173, 2787, 4259, 822, 336, 559]
Built_Areas = [19704, 20971, 21196, 21533, 21820, 21236, 21360]
Bare_Ground_Areas = [317, 66, 23, 0, 22, 75, 238]
Rangelands_Areas = [11668, 14502, 14818, 13077, 15813, 13176, 16809]

# Combine data into a DataFrame
data = {
    'Year': Year,
    'WA': Water_Areas,
    'TR': Trees,
    'FVA': Flooded_Vegetation_Areas,
    'CR': Crops,
    'BA': Built_Areas,
    'BGA': Bare_Ground_Areas,
    'RA': Rangelands_Areas
}
df = pd.DataFrame(data)

# Normalize the data to make the total length of each bar the same
df_normalized = df.iloc[:, 1:].div(df.iloc[:, 1:].sum(axis=1), axis=0)

# Scale smaller categories to make them more visible
df_normalized['WA'] *= 10
df_normalized['FVA'] *= 10
df_normalized['CR'] *= 10
df_normalized['BGA'] *= 10

# Re-normalize after scaling
df_normalized = df_normalized.div(df_normalized.sum(axis=1), axis=0)

# Prepare data for Sankey diagram
labels = []
source = []
target = []
value = []

# Define categories and colors (in RGB format)
categories = df_normalized.columns
colors = [
    (0, 112, 255),    # Blue (Water)
    (38, 115, 0),    # Green (Trees)
    (85, 255, 0),  # Cyan (Flooded Vegetation)
    (255, 255, 0),  # Yellow (Crops)
    (255, 0, 0),    # Red (Built Areas)
    (255, 170, 0),# Gray (Bare Ground)
    (223, 115, 255)   # Orange (Rangelands)
]

# Create labels for each year and category
for i, year in enumerate(Year):
    for j, category in enumerate(categories):
        labels.append(f"{year} {category}")

# Create transitions between years
for i in range(len(Year) - 1):
    for j, category_from in enumerate(categories):
        for k, category_to in enumerate(categories):
            # Calculate the transition value as the minimum of the two proportions
            transition_value = min(df_normalized.iloc[i, j], df_normalized.iloc[i + 1, k])
            if transition_value > 0:
                source.append(i * len(categories) + j)
                target.append((i + 1) * len(categories) + k)
                value.append(transition_value)

# Normalize transitions to ensure uniform column heights
total_transitions = sum(value)
value = [v / total_transitions for v in value]

# Create Sankey diagram
fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color=[f"rgba({colors[i % len(categories)][0]}, "
               f"{colors[i % len(categories)][1]}, "
               f"{colors[i % len(categories)][2]}, 1)"
               for i in range(len(labels))],
        # Align the top of the columns
        y=[0.9] * len(labels)  # Align all nodes to the top
    ),
    link=dict(
        source=source,
        target=target,
        value=value,
        color=[f"rgba({colors[k % len(categories)][0]}, "
               f"{colors[k % len(categories)][1]}, "
               f"{colors[k % len(categories)][2]}, 0.3)"  # 50% transparency
               for k in target]
    )
))

# Customize layout
fig.update_layout(
    title="Land Use Transitions (2017~2023)",
    font=dict(size=14, family="Times New Roman"),
    height=800,
    width=1200,
    title_x=0.066,
    title_y=0.91  # Adjust the title to be closer to the plot
)

# Show the plot
fig.show()
# 保存720dpi的图片
fig.write_image("land_use_transitions.png", scale=4, engine="kaleido")